In [1]:
# !pip install -U geemap
# !pip install geedim
# !pip install mss

In [2]:
import os
import ee
import geemap

import csv

ee.Initialize()

In [3]:
INPUT_FILE = "all_known_crematoria_raj_lat_long_formatted.csv"
csv_reader = csv.reader(open(INPUT_FILE))

OUT_DIR = os.path.expanduser('.')
RADIUS = 500

VIS_PARAMS = {
    'bands': ['B6', 'B5', 'B4'],
    'min': 0,
    'max': 6000,
    'gamma': 1.4,
}

MAP = geemap.Map()

In [4]:
def getImgForAoi(aoi, name):
    collection = (
        ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
        .filterBounds(aoi.geometry())
        .sort("CLOUD_COVER")
        .limit(1)
    )
    image = collection.first()
    return image.set({
        'aoi': aoi.geometry(),
        'name': name
    }).clip(aoi.geometry())

In [5]:
def downloadImg(image):    
    out_img = os.path.expanduser(f"./landsat_{image.get('name')}.png")
    geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
    geemap.show_image(out_img)

In [6]:
aois = []
bboxes = []
images = []
names_collection = []

for l in csv_reader:
    if l[0] == 'name':
        continue

    point_geom = ee.Geometry.Point([float(l[2]), float(l[1])])
#     point_geom = ee.Geometry.Point([80.968294, 26.85694])
    aois.append(point_geom)
    
    bbox = ee.Feature(point_geom.buffer(RADIUS).bounds())
    bboxes.append(bbox)
    
    image = getImgForAoi(bbox, l[0])
    images.append(image)
    
    names_collection.append(l[0])
    
aoi_collection = ee.FeatureCollection(aois)
bboxes_collection = ee.FeatureCollection(bboxes)
images_collection = ee.ImageCollection(images)

In [7]:
MAP.addLayer(images_collection, VIS_PARAMS, "LANDSAT 7")
MAP

Map(center=[20, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(Togg…

In [8]:
# image = image.clip(bboxes[0].geometry())
# vis_params = {
#     'bands': ['B6', 'B5', 'B4'],
#     'min': 0,
#     'max': 6000,
#     'gamma': 1.4,
# }

# Map.addLayer(image, vis_params, "LANDSAT 7")
# # Map.setCenter(80.968294, 26.85694, 14)
# Map

# out_img = os.path.expanduser("./landsat.png")
# geemap.get_image_thumbnail(image, out_img, vis_params, dimensions=500, format='png')
# geemap.show_image(out_img)

In [9]:
out_dir = os.path.expanduser(".")
geemap.get_image_collection_thumbnails(
    images_collection, out_dir, VIS_PARAMS, dimensions=500, format="jpg",
    names = names_collection
)

Total number of images: 39

